In [2]:
import matplotlib.pyplot as plt
import random
import sys
import math
from scipy.stats import bernoulli

# Problème du voyageur de commerce

In [3]:
indices = []
x = []
y = []
points = []
with open("RS/16.tsp", "r") as f:
    for line in f:
        line_list = line.strip().split(" ")
        points.append(line_list)
        indices.append(line_list[0])
        x.append(line_list[1])
        y.append(line_list[2])

In [4]:
points

[['1', '38.24', '20.42'],
 ['2', '39.57', '26.15'],
 ['3', '40.56', '25.32'],
 ['4', '36.26', '23.12'],
 ['5', '33.48', '10.54'],
 ['6', '37.56', '12.19'],
 ['7', '38.42', '13.11'],
 ['8', '37.52', '20.44'],
 ['9', '41.23', '9.10'],
 ['10', '41.17', '13.05'],
 ['11', '36.08', '-5.21'],
 ['12', '38.47', '15.13'],
 ['13', '38.15', '15.35'],
 ['14', '37.51', '15.17'],
 ['15', '35.49', '14.32'],
 ['16', '39.36', '19.56']]

In [5]:
def total_distance(path):
    dist_total = 0
    for i in range(len(path)):
        if i != len(path) - 1:
            dist_total += math.sqrt(pow(float(path[i][1]) - float(path[i + 1][1]),2) + pow(float(path[i][2]) - float(path[i + 1][2]), 2))
        else:
            dist_total += math.sqrt(pow(float(path[i][1]) - float(path[0][1]), 2) + pow(float(path[i][2]) - float(path[0][2]), 2))
    return(dist_total)

## Version naïve 

In [6]:
def algo_naif(points):
    dist_min = sys.maxsize
    path_min = []
    for i in range(1000 * len(points)):
        path = random.sample(points, len(points))
        dist_total = total_distance(path)
        if(dist_total < dist_min):
            dist_min = dist_total
            path_min = path
    return(path_min, dist_min)

path, dist = algo_naif(points)

In [7]:
dist

94.07923381706372

## Hill Climbing

In [8]:
def hill_climbing(points):

    dist_min = sys.maxsize

    for i in range(1000 * len(points)):

        path = []
        current_point = random.choice(points)
        
        while len(path) != len(points):
            dist_min_2_points = sys.maxsize
            nearest_neighbor = None
            for point in points:
                if not(point in path) and point != current_point:
                    dist = math.sqrt(pow(float(current_point[1]) - float(point[1]), 2) + pow(float(current_point[2]) - float(point[2]), 2))
                    if(dist < dist_min_2_points):
                        dist_min_2_points = dist
                        nearest_neighbor = point
                elif len(path) == len(points) - 1:
                    nearest_neighbor = current_point
            path.append(nearest_neighbor)

            
        dist_total = total_distance(path)
        if(dist_total < dist_min):
            dist_min = dist_total
            path_min = path
        
    
    return(path_min, dist_min)
                
path, dist = hill_climbing(points)

In [9]:
dist

88.82869495892582

## Coder l'algorithme

In [10]:
def disturbance(path):
    [indice1, indice2] = random.sample(range(len(path)), 2)
    tmp = path[indice1]
    path[indice1] = path[indice2]
    path[indice2] = tmp
    return(path)

path_disturbance = disturbance(path)

In [11]:
def metropolisCriteria(delta, t):
    if(delta < 0):
        res = True
    else :
        res_prob = bernoulli.rvs(math.exp(-delta/t))
        res = (res_prob == 1)
    return(res)

In [12]:
def cooling(alpha, t):
    return(alpha*t)

In [93]:
def recuit_simule(path, t):

    dist = total_distance(path)
    path_min = path
    dist_min = dist

    cpt = 1

    convergence_value_min = []

    while cpt < 2000:

        cpt_without_improvement = 1
        improved = False

        while cpt_without_improvement < 100 and not improved:

            neighbor = disturbance(path)
            dist_neighbor = total_distance(neighbor)
            delta = dist_neighbor - dist

            if metropolisCriteria(delta, t):

                path = neighbor
                dist = dist_neighbor
                
                if delta > 0 and dist_neighbor < dist_min:

                    dist_min = dist_neighbor
                    path_min = neighbor
                    improved = True
            
            cpt_without_improvement += 1
        
        cpt += 1

        t = cooling(0.99, t)

        convergence_value_min.append(dist_min)
    
    return(path_min, dist_min, convergence_value_min)
    

In [94]:
path, dist = algo_naif(points)
path_min, dist_min, convergence_value_min = recuit_simule(path.copy(), 10000)

In [95]:
dist_min

92.22384720146437